In [1]:
import preamble

# Constants

In [ ]:
import numpy as np

# Experiment
USER_ID = 0

# Trajectory Planner
REFERENCE_TRAJECTORY_CSV_PATH = '../data/reference_path.csv'
INIT_POSE = np.array([-0.53, 0.28, 0.10, -1.16, -1.19, 1.18]) # Also used for Robot

# Verbal PD
K_v = 12.3
B_v = 11.4

# F2L
T_proj = 4
N_proj = 256
dt_proj = T_proj / N_proj

# Physical PD
K_p = 17.6
B_p = 11.5

# Physical Admittance
M = 2
B = 17.7

# F2L
F2L_MODEL_PATH = '../data/F2L_models/svm_knn.pkl'
FINAL_IMPULSE_THRESHOLD = 0

# Real time data collection
TABLE_COLUMN_NAMES = [
    't',
    'x',
    'x_dot',
    'F_h',
    'x_ref',
    'x_dot_ref',
    'e',
    'e_dot',
    'P_hat',
    'V_hat',
    'c_p',
    'c_v',
    'verbal_P_tracking_term',
    'verbal_D_tracking_term',
    'verbal_PD_tracking_term',
    'A_v',
    'U_v',
    'physical_P_tracking_term',
    'physical_D_tracking_term',
    'physical_PD_tracking_term',
    'A_p',
    'U_p',
    'phrase',
    'U',
    'human_disturbance',
    'system_dynamics_input',
]
DATA_COLLECTION_SAMPLING_RATE = 200

# Real time data plotting
PLOTTING_SAMPLING_RATE = 30

# Robot
ROBOT_IP = '169.254.9.43'
FORCE_EWMA_TAU = 0.5
TRANSLATIONAL_FORCE_DEADBAND = 2

# Experiment

This baseline is made up of all componentes: Physical PD, Physical Admittance, Verbal PD, Verbal Admittance, F2L, Compliance Estimator, and $J$ Optimization Method.

In [ ]:
from timer import Timer
from trajectory_planner import TrajectoryPlanner
from model import SVMKNNModel
import pickle
from vocalizer import Vocalizer
from virtual_dynamics import SimpleVirtualDynamics
from tabular_data_store import TabularDataStore
from plot_client import PlotClient
from robot import Robot
from console_command_thread import ConsoleCommandThread
import scipy.stats
from phrase_mapping import PHRASE_MAPPING, MEAN_WORDS_PER_PHRASE

#############
### SETUP ###
#############

### Experiment ###
experiment_timer = Timer()

### Trajectory Planner ###
traj = TrajectoryPlanner(REFERENCE_TRAJECTORY_CSV_PATH, INIT_POSE[:3])

### F2L ###
with open(F2L_MODEL_PATH, 'rb') as file:
    F2L_model: SVMKNNModel = pickle.load(file)
phrase = ''
voc = Vocalizer()
speaking_timer = Timer()

### System dynamics ###
vd = SimpleVirtualDynamics(M=M, B=B, K=0)

### Real time data collection ###
table = TabularDataStore(column_names=TABLE_COLUMN_NAMES)
table_timer = Timer()

### Real time data plotting ###
plot = PlotClient()

plot.create_plot("force_plot", title="Force vs Time", xlabel="Time (s)", ylabel="Force")
plot.create_line("force_plot", "Fx", color="r", label="Fx")
plot.create_line("force_plot", "Fy", color="g", label="Fy")
plot.create_line("force_plot", "Fz", color="b", label="Fz")

plot.create_plot("velocity_plot", title="Velocity vs Time", xlabel="Time (s)", ylabel="Velocity")
plot.create_line("velocity_plot", "vx", color="r", label="vx")
plot.create_line("velocity_plot", "vy", color="g", label="vy")
plot.create_line("velocity_plot", "vz", color="b", label="vz")

plot.create_plot("error_plot", title="Error vs Time", xlabel="Time (s)", ylabel="Error")
plot.create_line("error_plot", "ex", color="r", label="ex")
plot.create_line("error_plot", "ey", color="g", label="ey")
plot.create_line("error_plot", "ez", color="b", label="ez")
 
plot.create_plot("control_plot", title="Control Inputs vs Time", xlabel="Time (s)", ylabel="Control Norm")
plot.create_line("control_plot", "U_p", color="cyan", label="‖U_p‖")
plot.create_line("control_plot", "U_v", color="magenta", label="‖U_v‖")

plot.create_plot("U_v_projection_plot", title="Projected U_v Profile", xlabel="Future Steps", ylabel="U_v")
plot.create_line("U_v_projection_plot", "U_v_x", color="r", label="U_v,x")
plot.create_line("U_v_projection_plot", "U_v_y", color="g", label="U_v,y")
plot.create_line("U_v_projection_plot", "U_v_z", color="b", label="U_v,z")

plot_timer = Timer()

### Robot ###
with Robot(
    ROBOT_IP,
    default_axes=Robot.TRANSLATION,
    init_pose=INIT_POSE,
    force_ewma_tau=FORCE_EWMA_TAU,
    translational_force_deadband=TRANSLATIONAL_FORCE_DEADBAND,
) as r, ConsoleCommandThread() as c:

    ##################
    ### EXPERIMENT ###
    ##################

    experiment_timer.reset()
    speaking_timer.reset()

    while c.is_alive():
        period_start = r.control.initPeriod()

        ### Current state ###
        t = experiment_timer.t()
        dt = experiment_timer.dt()
        x = r.get_pose()
        x_dot = r.get_velocity()
        F_h = r.get_force()

        ### Trajectory Planner ###
        traj.update_reference_trajectory(x)
        x_ref, x_dot_ref = traj.get_closest_target(x)

        ### Tracking errors ###
        e = x_ref - x
        e_dot = x_dot_ref - x_dot

        ### Compliance Estimator ###
        P_hat = 0.5
        V_hat = 0.5

        ### Guidance costs ###
        c_p = (P_hat + V_hat) / 2
        c_v = 1 - V_hat

        ### Verbal PD ###
        verbal_P_tracking_term = K_v * e
        verbal_D_tracking_term = B_v * e_dot
        verbal_PD_tracking_term = verbal_P_tracking_term + verbal_D_tracking_term

        ### Verbal Admittance ###
        A_v = c_p / (c_p + c_v)
        U_v = verbal_PD_tracking_term - A_v*F_h

        ### Physical PD ###
        physical_P_tracking_term = K_p * e
        physical_D_tracking_term = B_p * e_dot
        physical_PD_tracking_term = physical_P_tracking_term + physical_D_tracking_term

        ### Physical Admittance ###
        A_p = c_v / (c_p + c_v)
        U_p = physical_PD_tracking_term - A_p*F_h

        ### F2L ###
        x_sim = np.copy(x)
        x_dot_sim = np.copy(x_dot)

        U_v_profile = []

        for _ in range(N_proj):
            x_ref_proj, x_dot_ref_proj = traj.get_closest_target(x_sim)
            
            e_proj = x_ref_proj - x_sim
            e_dot_proj = x_dot_ref_proj - x_dot_sim

            U_v_proj = (K_v * e_proj + B_v * e_dot_proj) / (1 + A_v)
            U_v_profile.append(U_v_proj)

            U_p_proj = K_p * e_proj + B_p * e_dot_proj - A_p * U_v_proj

            U_proj = U_p_proj + U_v_proj

            a_proj = (U_proj - B * x_dot_sim) / M
            x_dot_sim += a_proj * dt_proj
            x_sim += x_dot_sim * dt_proj

        U_v_profile = np.array(U_v_profile)
        impulse_curve = U_v_profile.cumsum(axis=0) * dt_proj
        final_impuse = impulse_curve[-1, :]

        speaking_period = MEAN_WORDS_PER_PHRASE / (0.36 * 2.2 ** (2 - P_hat - V_hat))

        if not voc.is_uttering():
            if np.linalg.norm(final_impuse) < FINAL_IMPULSE_THRESHOLD or speaking_timer.t() <= speaking_period:
                phrase = ''
            else:
                words = F2L_model.force_to_phrase(impulse_curve[None, :])[0]
                phrase = PHRASE_MAPPING[' '.join(words).strip()]
                speaking_timer.reset()

        did_utter = voc.utter(phrase)

        is_speaking = voc.is_uttering()

        ### System dynamics ###
        U = U_p + U_v
        human_disturbance = F_h - U_v
        system_dynamics_input = U + human_disturbance
        vd.apply_force(system_dynamics_input, dt)
        r.set_velocity(vd.get_velocity(), acceleration=10)

        ### Real time data collection ###
        if table_timer.t() >= DATA_COLLECTION_SAMPLING_RATE**(-1):
            table.append_row((
                t,
                x,
                x_dot,
                F_h,
                x_ref,
                x_dot_ref,
                e,
                e_dot,
                P_hat,
                V_hat,
                c_p,
                c_v,
                verbal_P_tracking_term,
                verbal_D_tracking_term,
                verbal_PD_tracking_term,
                A_v,
                U_v,
                physical_P_tracking_term,
                physical_D_tracking_term,
                physical_PD_tracking_term,
                A_p,
                U_p,
                phrase,
                U,
                human_disturbance,
                system_dynamics_input,
            ))
            
            table_timer.reset()
            
        ### Real time data plotting ###
        if plot_timer.t() >= PLOTTING_SAMPLING_RATE**(-1):
            plot.update_line("force_plot", "Fx", (t, F_h[0]))
            plot.update_line("force_plot", "Fy", (t, F_h[1]))
            plot.update_line("force_plot", "Fz", (t, F_h[2]))

            plot.update_line("velocity_plot", "vx", (t, x_dot[0]))
            plot.update_line("velocity_plot", "vy", (t, x_dot[1]))
            plot.update_line("velocity_plot", "vz", (t, x_dot[2]))

            plot.update_line("error_plot", "ex", (t, e[0]))
            plot.update_line("error_plot", "ey", (t, e[1]))
            plot.update_line("error_plot", "ez", (t, e[2]))

            plot.update_line("compliance_plot", "P_hat", (t, P_hat))
            plot.update_line("compliance_plot", "V_hat", (t, V_hat))

            plot.update_line("control_plot", "U_p", (t, np.linalg.norm(U_p)))
            plot.update_line("control_plot", "U_v", (t, np.linalg.norm(U_v)))

            timesteps = np.arange(len(U_v_profile)) * dt_proj

            plot.update_line("U_v_projection_plot", "proj_v_impulse_x", (timesteps, impulse_curve[:, 0]), mode="replace")
            plot.update_line("U_v_projection_plot", "proj_v_impulse_y", (timesteps, impulse_curve[:, 1]), mode="replace")
            plot.update_line("U_v_projection_plot", "proj_v_impulse_z", (timesteps, impulse_curve[:, 2]), mode="replace")

            plot_timer.reset()

        r.control.waitPeriod(period_start)

# table.to_pandas().to_pickle(f'../data/experiments/physical_only/{USER_ID}.pkl')